# Kaggle Titanic survival - logistic regression model

This is an introduction to machine learning classification. Much more training is available at:

https://pythonhealthcare.org/titanic-survival/

Can we predict which passengers would survive the sinking of the Titanic?

See:

https://www.kaggle.com/c/titanic/overview/evaluation

https://gitlab.com/michaelallen1966/1908_coding_club_kaggle_titanic

The original data comes from:

https://www.kaggle.com/c/titanic/data

Though we will download and use a data set that has been pre-processed ready for machine learning.

The data includes:

Variable  | Definition
----------|-----------
survival  | Survival (0 = No, 1 = Yes)
pclass    | Ticket class
sex       | Sex
Age       | Age in years
sibsp     | # of siblings / spouses aboard the Titanic
parch     | # of parents / children aboard the Titanic
ticket    | Ticket number
fare      | Passenger fare
cabin     | Cabin number
embarked  | Port of Embarkation(C=Cherbourg, Q=Queenstown, S=Southampton)

## Logistic regression

In this example we will use logistic regression (see https://en.wikipedia.org/wiki/Logistic_regression).

For an introductory video on logistic regression see: https://www.youtube.com/watch?v=yIYKR4sgzI8

Logistic regression takes a range of features (which we will normalise/standardise to put on the same scale) and returns a probability that a certain classification (survival in this case) is true.

We will go through the following steps:

* Download and save pre-processed data
* Split data into features (X) and label (y)
* Split data into training and test sets (we will test on data that has not been used to fit the model)
* Standardise data
* Fit a logistic regression model (from sklearn)
* Predict survival of the test set, and assess accuracy
* Review model coefficients (weights) to see importance of features
* Show probability of survival for passengers

## Load modules

A standard Anaconda install of Python (https://www.anaconda.com/distribution/) contains all the necessary modules.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Import machine learning methods
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Download data if required

The section below downloads pre-processed data, and saves it to a subfolder (from where this code is run).
If data has already been downloaded that cell may be skipped.

Code that was used to pre-process the data ready for machine learning may be found at:

https://github.com/MichaelAllen1966/1804_python_healthcare/blob/master/titanic/01_preprocessing.ipynb

In [ ]:
def download_data():
    """Download data from web"""
    
    address = 'https://raw.githubusercontent.com/MichaelAllen1966/' + \
                '1804_python_healthcare/master/titanic/data/processed_data.csv'
    
    df = pd.read_csv(address)

    # Create a data subfolder if one does not already exist
    import os
    data_directory ='./data/'
    if not os.path.exists(data_directory):
        os.makedirs(data_directory)

    # Save data
    df.to_csv(data_directory + 'processed_data.csv', index=False)

In [ ]:
#download_data()

In [ ]:
def load_data():
    """Load data from csv into Pandas dataframe, and cast all data to float"""
    data = pd.read_csv('data/processed_data.csv')
    # Make all data 'float' type
    data = data.astype(float)
    return data

In [ ]:
data = load_data()

## Examine loaded data

The data is in the form of a Pandas DataFrame, so we have column headers providing information of what is contained in each column.

We will use the DataFrame `.head()` method to show the first few rows of the imported DataFrame. By default this shows the first 5 rows. Here we will look at the first 10.

In [ ]:
data.head(10)

We can also show a summary of the data with the `.describe()` method.

In [ ]:
data.describe()

The first column is a passenger index number. We will remove this, as this is not part of the original Titanic passenger data.

In [ ]:
def drop_passenger_id(df):
    """
    Takes a pandas dataframe (df)
    Drop passenger ID as it is not original data."
    inplace means to make change in table directly.
    axis=1 shows we are remocing a column (axis=0 would be row).
    Function returns uncganged datadrame if PassengerId does not exist
    
    """
    if 'PassengerId' in list(df):
        df.drop('PassengerId', inplace=True, axis=1)
        return df
    else:
        return df

In [ ]:
data = drop_passenger_id(data)

## Looking at a summary of passengers who survived or did not survive

Before running machine learning models, it is always good to have a look at your data. Here we will separate passengers who survived from those who died, and we will have a look at differences in features.

We will use a *mask* to select and filter passengers.

In [ ]:
mask = data['Survived'] == 1 # Mask for passengers who survive
survived = data[mask] # filter using mask

mask = data['Survived'] == 0 # Mask for passengers who died
died = data[mask] # filter using mask

Now let's look at average (mean) values for `survived` and `died`.

In [ ]:
survived.mean()

In [ ]:
died.mean()

We can make looking at them side by side more easy by putting these values in a new DataFrame.

In [ ]:
summary = pd.DataFrame() # New empty DataFrame
summary['survived'] = survived.mean()
summary['died'] = died.mean()

Now let's look at them side by side. See if you can spot what features you think might have influenced survival.

In [ ]:
summary

## Divide into X (features) and y (labels)

We will separate out our features (the data we use to make a prediction) from our label (what we are truing to predict).
By convention our features are called `X` (usually upper case to denote multiple features), and the label (survived or not) `y`.

In [ ]:
def get_features_and_label(df):
    """
    Produces features by dropping survived column.
    Produces labels by retuening just the survived column
    """
    X = df.drop('Survived',axis=1)
    y = df['Survived']
    
    return X, y

In [ ]:
X, y = get_features_and_label(data)

## Divide into training and tets sets

When we test a machine learning model we should always test it on data that has not been used to train the model.
We will use sklearn's `train_test_split` method to randomly split the data: 75% for training, and 25% for testing.

In [ ]:
def get_training_and_test_sets(X, y):
    """
    Use sklearn's train_test_split to randomly split data.
    75% training, 25% test
    """
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_training_and_test_sets(X,y)

## Standardise data

We want all of out features to be on roughly the same scale. This generally leads to a better model, and also allows us to more easily compare the importance of different features.

One simple method is to scale all features 0-1 (by subtracting the minimum value for each value, and dividing by the new remaining maximum value).

But a more common method used in many machine learning methods is standardisation, where we use the mean and standard deviation of the training set of data to normalise the data. We subtract the mean of the test set values, and divide by the standard deviation of the training data. Note that the mean and standard deviation of the training data are used to standardise the test set data as well.

Here we will use sklearn's `StandardScaler method`. This method also copes with problems we might otherwise have (such as if one feature has zero standard deviation in the training set).

In [ ]:
def standardise_data(X_train, X_test):
    """
    Standardise data, by subtracting the feature mean, and dividing by
    the feature standard deviation. 
    
    Uses sklearn's StandardScalar
    """
    
    # Initialise a new scaling object for normalising input data
    sc = StandardScaler() 

    # Set up the scaler just on the training set
    sc.fit(X_train)

    # Apply the scaler to the training and test sets
    train_std=sc.transform(X_train)
    test_std=sc.transform(X_test)
    
    return train_std, test_std

In [ ]:
X_train_std, X_test_std = standardise_data(X_train, X_test)

In [ ]:
X_test

## Define function to calculate accuracy, sensitivity and specificity

* *Sensitivity*: the proportion of positive label (survivors) currectly identified (also called recall)
* *Specificity*: the proportion of negative label (non-survivors) currectly identified

In [ ]:
def calculate_accuracy(observed, predicted):
    
    """
    Calculates accuracy, sensitivity, and specificity.
    Returns a dictionary of results
    """
    
    # Converts list to NumPy arrays (if needed)
    if type(observed) == list:
        observed = np.array(observed)
    if type(predicted) == list:
        predicted = np.array(predicted)
        
    # calculate simple accuracy
    accuracy = np.mean(observed == predicted)
    
    # Split into observed and predicted negatives and positives
    observed_positives = observed == 1
    observed_negatives = observed == 0
    predicted_positives = predicted == 1
    predicted_negatives = predicted == 0
    
    # Calculate true and false positive/negatives
    # We calaculate mroe than we need here, but they are useful to put here
    true_positives = (predicted_positives == 1) & (observed_positives == 1)
    false_positives = (predicted_positives == 1) & (observed_positives == 0)
    true_negatives = (predicted_negatives == 1) & (observed_negatives == 1)
    false_negatives = (predicted_negatives == 1) & (observed_negatives == 0)
    
    # Calculate sensitivity and specificity
    sensitivity = np.sum(true_positives) / np.sum(observed_positives)
    specificity = np.sum(true_negatives) / np.sum(observed_negatives)
    
    # Put results in a dictionary (keyword and value)    
    results = {'accuracy': accuracy,
               'sensitivity': sensitivity,
               'specificity': specificity}
    
    return results

## Fit logistic regression model

Now we will fit a logistic regression model, using sklearn's `LogisticRegression` method. Our machine learning model fitting is only two lines of code!
By using the name `model` for our logistic regression model we will make our model more interchangeable later on.

In [ ]:
def build_lr_model(X_train_std,y_train):
    """Set up sklearn logistic regression model and train"""

    model = LogisticRegression()
    model.fit(X_train_std,y_train)
    
    return model

In [ ]:
model = build_lr_model(X_train_std,y_train)
# Show model
model

## Predict values

Now we can use the trained model to predict survival. We will test the accuracy of both the training and test data sets.

In [ ]:
def predict_categories(model, X):
    """Predict label categories, given features X"""

    return model.predict(X)

In [ ]:
y_pred_train = predict_categories(model, X_train_std)
y_pred_test = predict_categories(model, X_test_std)

## Calculate accuracy

In [ ]:
results = calculate_accuracy(y_test, y_pred_test)

# print results using python print format to round decimal places

print ('Accuracy: {0:0.2f}'.format(results['accuracy']))
print ('Sensitivity: {0:0.2f}'.format(results['sensitivity']))
print ('Specificity: {0:0.2f}'.format(results['specificity']))

Not bad. 

## Examining the model coefficients (weights)

Not all features are equally important. And some may be of little or no use at all, unnecessarily increasing the complexity of the model. In a later notebook we will look at selecting features which add value to the model (or removing features that don’t).

Here we will look at the importance of features – how they affect our estimation of survival. These are known as the model *coefficients* (if you come from a traditional statistics background), or model *weights* (if you come from a machine learning background). 

Because we have standardised our input data the magnitude of the weights may be compared as an indicator of their influence in the model. Weights with higher negative numbers mean that that feature correlates with reduced chance of survival. Weights with higher positive numbers mean that that feature correlates with increased chance of survival. Those weights with values closer to zero (either positive or negative) have less influence in the model.

We access the model weights my examining the model `coef_` attribute. The model may predict more than one outcome label, in which case we have weights for each label. Because we are predicting a signle label (survive or not), the weights are found in the first element (`[0]`) of the `coef_` attribute.

In [ ]:
model.coef_[0]

So we have an array of model weights.

Not very readable for us mere humans is it?!

We will write a function to transfer the weights array to a Pandas DataFrame. The array order is in the same order of the list of features of X, so we will put that those into the DataFrame as well. And we will sort by influence in the model. Because both large negative and positive values are more influential in the model we will take the *absolute* value of the weight (that is remove any negative sign), and then sort by that absolute value. That will give us a more readable table of most influential features in the model.

In [ ]:
def get_model_weights(model, X):
    """Returns a sorted dataframe of model weights. Assumes a single class.
    Takes model and X dataframe"""
    
    co_eff_df = pd.DataFrame() # create empty DataFrame
    
    # Get list of features
    co_eff_df['feature'] = list(X) # Get feature names from X
    
    # Get weights (for one class)
    co_eff_df['co_eff'] = model.coef_[0]
    
    # Get absolute value of weight and sort by absolute value
    co_eff_df['abs_co_eff'] = np.abs(co_eff_df['co_eff'])
    co_eff_df.sort_values(by='abs_co_eff', ascending=False, inplace=True)

    return co_eff_df

In [ ]:
model_weights = get_model_weights(model, X)
print (model_weights)

## Show predicted probabilities

The predicted probabilities are for the two alternative classes 0 (does not survive) or 1 (survive).

Ordinarily we do not see these probabilities - the `predict` method used above applies a cut-off of 0.5 to classify passengers into survived or not, but we can see the individual probabilities for each passenger.

Later we will use these to adjust sensitivity of our model to detecting survivors or non-survivors.

Each passenger has two values. These are the probability of not surviving (first value) or surviving (second value). Because we only have two possible classes we only need to look at one. Multiple values are important when there are more than one class being predicted.

In [ ]:
model.predict_proba(X_test_std)[0:10]

Let's wrapi it in a function and show just the probability of the positive class.

In [ ]:
def predict_probabilities(model, X):
    """Predict label categories, given features X"""
    
    probabilities = model.predict_proba(X)
    prob_of_positive_class = probabilities[:,1]

    return prob_of_positive_class

In [ ]:
# Show first ten predicted probabilities 
# (note how the values relate to the classes predicted above)
probabilities = predict_probabilities(model, X_test_std)
probabilities[0:10]

If we compare this to the standard classification, we'll see that everything with a probability over 0.5 will be classed as positive.

We'll put it in a dataframe to comapre more easily by eye:

In [ ]:
# Show first ten predicted classes

df = pd.DataFrame()
df['probabilty'] = predict_probabilities(model, X_test_std)
df['class'] = predict_categories(model, X_test_std)

df.head(10)

Let's check that the number of postive classes using the standard classification method is the same as the number of cases that have a probability of >= 0.5 (the print formating prints with zero decimal places).

In [ ]:
print ('Number of positives: {0:.0f}'.format(np.sum(df['class']==1)))
print ('Number of prob >= 0.5: {0:.0f}'.format(np.sum(df['probabilty']>=0.5)))

## Changing classification threshold

We can use the model probability output to change the sensitivity of the model. Reducing the threshold will increase the sensitivity of the model (the ability to detect positives), but will reduce the specificity of the model (he ability to detect negatives). Depending on which is most important (the ability to identify positive cases, or the importance in not misclassifying negative cases), we can fine-tune our model in the way we wish.

Let us put classification with a given threshold into a function. We will seta default threshold of 0.5 so our function will beahve like a normal classifyer if no adjusted threshold is passed to it.

In [ ]:
def classify_with_threshold(model, X, threshold=0.5):
    """Predict label categories, given features X, and an optional threshold """
    
    probabilities = model.predict_proba(X)
    positives = probabilities[:,1] >= threshold
    
    return positives

Lets' test it at a couple of thresholds (default, and a reduce threshold).

In [ ]:
y_pred_test = classify_with_threshold(model, X_test_std)

results = calculate_accuracy(y_test, y_pred_test)

# print results using python print format to round decimal places

print ('Accuracy: {0:0.2f}'.format(results['accuracy']))
print ('Sensitivity: {0:0.2f}'.format(results['sensitivity']))
print ('Specificity: {0:0.2f}'.format(results['specificity']))

In [ ]:
y_pred_test = classify_with_threshold(model, X_test_std, threshold=0.2)

results = calculate_accuracy(y_test, y_pred_test)

# print results using python print format to round decimal places

print ('Accuracy: {0:0.2f}'.format(results['accuracy']))
print ('Sensitivity: {0:0.2f}'.format(results['sensitivity']))
print ('Specificity: {0:0.2f}'.format(results['specificity']))

## Putting it all together

We've defined a lot of re-usable functions. This allows us to put together compact code for the whole pipeline.

In [ ]:
data = load_data()
data = drop_passenger_id(data)
X, y = get_features_and_label(data)
X_train_std, X_test_std = standardise_data(X_train, X_test)
model = build_lr_model(X_train_std,y_train)
threshold = 0.5
y_pred_test = classify_with_threshold(model, X_test_std, threshold=threshold)
results = calculate_accuracy(y_test, y_pred_test)

# print results using python print format to round decimal places

print ('Accuracy: {0:0.2f}'.format(results['accuracy']))
print ('Sensitivity: {0:0.2f}'.format(results['sensitivity']))
print ('Specificity: {0:0.2f}'.format(results['specificity']))

# Print first 10 classifications
print('\nFirst 10 classifications of test set:')
print(y_pred_test[0:10])

## Excercise

Using the functions we have defined, create a chart of how accuracy, sensitivity and specificity change with varying thresholds.

You can reveal the answer below (you may have done it a different way).

In [ ]:
# write you code here

Reveal answer using the small dots

In [ ]:
# Load and split data
data = load_data()
data = drop_passenger_id(data)
X, y = get_features_and_label(data)

# Standardise data
X_train_std, X_test_std = standardise_data(X_train, X_test)

# Build model
model = build_lr_model(X_train_std,y_train)

# Loop through a list of thresholds
thresholds = np.arange(0, 1.01, 0.05)
results_accuracy = []
results_sensitivity = []
results_specificity = []

for threshold in thresholds:
    # Classify using threshold
    y_pred_test = classify_with_threshold(
        model, X_test_std, threshold=threshold)
    
    # Get results and append to lists
    results = calculate_accuracy(y_test, y_pred_test)
    results_accuracy.append(results['accuracy'])
    results_sensitivity.append(results['sensitivity'])
    results_specificity.append(results['specificity'])
    
# Define a plot function
def plot_results(thresholds, accuracy, sensitivity, specificity):
    """Create line plot of the relationship between classification threshold and
    accuracy, sensitivity, and specificity"""
    
    plt.plot(thresholds, accuracy, label='accuracy', marker='s')
    plt.plot(thresholds, sensitivity, label='sensitivity', marker='o')
    plt.plot(thresholds, specificity, label='specificity', marker='^')
    plt.xlabel('Threshold')
    plt.ylabel('Result')
    plt.legend()
    plt.show()

# Call plot function    
plot_results(thresholds, results_accuracy, results_sensitivity, 
             results_specificity)